In [1]:
import os
import glob
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

In [2]:
random.seed(1)
tf.random.set_seed(1)
np.random.seed(1)
plt.rcParams.update({'font.size': 18})

In [3]:
base_dir = './data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
base_dir, train_dir, validation_dir, test_dir

('./data', './data\\train', './data\\validation', './data\\test')

In [4]:
('./data',
 './data/train',
 './data/validation',
 './data/test')

('./data', './data/train', './data/validation', './data/test')

In [5]:
train_generator = ImageDataGenerator(rescale=1/255)
validation_generator = ImageDataGenerator(rescale=1/255) 

train_dataset = train_generator.flow_from_directory(batch_size=64,
                                    directory=train_dir,
                                    shuffle=True,
                                    target_size=(150, 150),
                                    class_mode='categorical')

validation_dataset = validation_generator.flow_from_directory(batch_size=64,
                                                         directory=validation_dir,
                                                         shuffle=True,
                                                         target_size=(150, 150),
                                                         class_mode='categorical')

Found 32578 images belonging to 6 classes.
Found 8473 images belonging to 6 classes.


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3), name='conv1'),
    tf.keras.layers.MaxPooling2D(2, 2, name='pooling1'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', name='conv2'),
    tf.keras.layers.MaxPooling2D(2, 2, name='pooling2'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', name='conv3'),
    tf.keras.layers.MaxPooling2D(2, 2, name='pooling3'),
    tf.keras.layers.Flatten(name='flatten1'),
    tf.keras.layers.Dense(1024, activation='relu', name='dense1'),
    tf.keras.layers.Dense(6, activation='softmax', name='dense2')
])

In [7]:
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 148, 148, 128)     3584      
                                                                 
 pooling1 (MaxPooling2D)     (None, 74, 74, 128)       0         
                                                                 
 conv2 (Conv2D)              (None, 72, 72, 32)        36896     
                                                                 
 pooling2 (MaxPooling2D)     (None, 36, 36, 32)        0         
                                                                 
 conv3 (Conv2D)              (None, 34, 34, 64)        18496     
                                                                 
 pooling3 (MaxPooling2D)     (None, 17, 17, 64)        0         
                                                                 
 flatten1 (Flatten)          (None, 18496)             0

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
with tf.device("/device:CPU:0"):
    history = model.fit(train_dataset, validation_data=validation_dataset, epochs=50, validation_steps=5, callbacks=[early_stopping], verbose=1)

Epoch 1/50
 77/510 [===>..........................] - ETA: 16:15 - loss: 1.3964 - acc: 0.5085

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = (12, 4)

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(history.history['acc'], label='Training Accuracy')
ax1.plot(history.history['val_acc'], label='Validation Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0.0, 1.1)
ax1.legend()

ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.legend()

plt.show()

In [ ]:
test_images = glob.glob("%s/*.jpg" % (test_dir))

cats = ['bird dropping', 'clean', 'cracked', 'dirty', 'dust', 'snow']
true = np.zeros(4349)
true[1066:2656] = 1
true[2656:2732] = 2
true[2732:2975] = 3
true[2975:3799] = 4
true[3799:4349] = 5
preds = []

for path in test_images:
    img = tf.keras.preprocessing.image.load_img( path, target_size=(150, 150))
    
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    score = tf.nn.softmax(predictions[0])
    
    class_pred = tf.math.argmax(score)
    preds.append(class_pred.numpy())

In [ ]:
print(true)
print(preds)

report = classification_report(true, preds, target_names=cats)
print(report)

In [ ]:
cm = confusion_matrix(true, preds, labels=[0,1,2,3,4,5])

fig, ax = plt.subplots(figsize=(8,8))

ax = sns.heatmap(cm, xticklabels=cats, yticklabels=cats, annot=True, fmt=".1f", cmap="coolwarm", cbar=False, ax=ax)

ax.set_ylabel("True")
ax.set_xlabel("Predict")

In [ ]:
model.save("classifying.h5")
print("Saved model to disk")

In [ ]:
real_dir = os.path.join(base_dir, 'real')
real_images = glob.glob("%s/*.jpg" % (real_dir))
cats = ['bird dropping', 'clean', 'cracked', 'dirty', 'dust', 'snow']
real_true = np.ones(len(real_images))
real_preds = []

for path in real_images:
    img = tf.keras.preprocessing.image.load_img( path, target_size=(150, 150))
    
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    score = tf.nn.softmax(predictions[0])
    
    class_pred = tf.math.argmax(score)
    real_preds.append(class_pred.numpy())


In [ ]:
print(real_true)
print(real_preds)

real_report = classification_report(real_true, real_preds)
print(real_report)

In [ ]:
real_cm = confusion_matrix(real_true, real_preds, labels=[0,1,2,3,4,5])
fig, ax = plt.subplots(figsize=(5,5))

ax = sns.heatmap(real_cm, xticklabels=cats, yticklabels=cats, annot=True, cmap="coolwarm", cbar=False, ax=ax)

ax.set_ylabel("True")
ax.set_xlabel("Predict")

In [ ]:
print(tf.__version__)

In [ ]:
import keras
print(tf.keras.__version__)